In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
import fitz
import pytesseract
import io
import PyPDF2

from pprint import pprint
from PIL import Image
from pdf2image import convert_from_path

import util
import lines
import group
import label
import records
import extract

In [181]:
test_files_dir = ".././test_files/"
test_dfs_dir = test_files_dir + "tesseract_data_frames/"
test_output_dir = test_files_dir + "extracted_indexes/"
test_files = util.list_files(test_files_dir)
test_dfs = util.list_files(test_dfs_dir)
file = test_dfs[-3]

indexes_path = "H:\Documents\Indexex\ILO_yearly_indices\\tesseract_data_frames"
indexes_csv = util.list_files(indexes_path, suffix=".csv")
output_dir = "H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes"

display(test_dfs, file)

['.././test_files/tesseract_data_frames/\\.~lock.LS_index_1976.csv#',
 '.././test_files/tesseract_data_frames/\\LS_index_1920.csv',
 '.././test_files/tesseract_data_frames/\\LS_index_1921.csv',
 '.././test_files/tesseract_data_frames/\\LS_index_1944.csv',
 '.././test_files/tesseract_data_frames/\\LS_index_1976.csv',
 '.././test_files/tesseract_data_frames/\\LS_index_1986.csv',
 '.././test_files/tesseract_data_frames/\\LS_index_1986_split.csv']

'.././test_files/tesseract_data_frames/\\LS_index_1976.csv'

In [182]:
#pdf_df = util.ocr(file, start_page=1, save_to=".././test_files")
pdf_df = pd.read_csv(file)
lines_df = lines.make_lines_df_from_ocr(pdf_df)
bins_x0, bins_x1, x0_n = group.group_line_starts_ends(lines_df)
borders = lines.make_borders_df(bins_x0, bins_x1)

lines_df

,line_text,x0,y0,x1,y1,page,dx
0,Hours of work: Arg. 1 (196-217); Bah. 1 (78-80...,320,278,3542,352,1,3222
1,Night work: Arg. 1 (200). Trade unions: Eg. 1;...,385,362,4449,427,1,4064
2,"Overtime: Arg. 1 (201, 203); Bah. 1.(79). (3-1...",386,416,3770,479,1,3384
3,Shift work: Arg. 1 (202).,388,481,857,524,1,469
4,Weekly rest: Arg. 1 (204-207); Bah. 1 (80); Ke...,388,528,2970,585,1,2582
...,...,...,...,...,...,...,...
1783,United Kingdom (UK),370,2499,785,2541,30,415
1784,1. (Great Britain) Trade unions and labour rel...,439,2553,1736,2600,30,1297
1785,2. (Great Britain) Race relations,435,2603,1068,2648,30,633
1786,International (Int.),372,2700,718,2741,30,346


In [214]:
def is_double_paged(pdf_df, borders):
    m = 100
    double_p = []
    
    for p, b in borders.groupby("page"):
        middle = b.iloc[0]["x0"] + b.iloc[0]["dx"]/2
        
        middle_words = pdf_df.loc[(pdf_df["page_num"]==p) & (pdf_df["left"] > middle-m) & (pdf_df["left"] < middle+m)]
        if middle_words.shape[0] <= 1:
            double_p.append(True)
        else:
            double_p.append(False)
            
    if double_p.count(True) > 0.8 * len(double_p):
        return True
    
    return False

In [226]:
if is_double_paged(pdf_df, borders):
    df = pdf_df.copy()
    pdf_l = pd.DataFrame()
    pdf_r = pd.DataFrame()
    
    for p, b in borders.groupby("page"):
        middle = b.iloc[0]["x0"] + b.iloc[0]["dx"]/2
        pdf_p = df.loc[df["page_num"] == p]
        
        l = pdf_p.loc[pdf_p["left"] <= middle]
        r = pdf_p.loc[pdf_p["left"] > middle]
        
        pdf_l = pd.concat([pdf_l, l])
        pdf_r = pd.concat([pdf_r, r])


rec_l = extract.extract_indexes(pdf_l)
rec_r = extract.extract_indexes(pdf_r)

idx_s = rec_l.shape[0]
idx_e = idx_s + rec_r.shape[0]
rec_r = rec_r.set_index(pd.Index(list(range(idx_s, idx_e))))

rec = pd.concat([rec_l, rec_r])
rec = rec.rename_axis("idx").sort_values(by=["page", "idx"])

In [227]:
rec.iloc[100:].head(20)

,country,region,text,page
idx,,,,
60,BELGIUM,,20 Feb. Royal Order to amend the Royal Order o...,6
61,BELGIUM,,27 Feb. An Act to amend the Roya | Order of 24...,6
62,BELGIUM,,22 Mar. Royal Order to amend the Royal Order o...,6
63,BELGIUM,,29 Mar. Act respecting family benefits for sel...,6
64,BELGIUM,,30 Mar. An Act respecting mea t te eco i ery [...,6
65,BELGIUM,,"26 Apr. Royal Order to amend ss. 122, 123, 125...",6
66,BELGIUM,,30 Apr. Royal Order respecting the right to an...,6
67,BELGIUM,,10 May Royal Order to amend the Royal Order of...,6
68,BELGIUM,,13 May An Act to amend the laws respecting ann...,6


In [10]:
#rec_df = extract.extract_indexes(pdf_df, save_to=test_output_dir + os.path.basename(file))
rec_df = extract.extract_indexes(pdf_df)
rec_df

Saved extracted indexes to .././test_files/extracted_indexes/LS_index_1921.csv.


,country,region,text,page
0,ENTERNATIONAL,,Feb. 9th. Convention respecting insurance agai...,3
1,ENTERNATIONAL,,Feb. 14th Convention between France and Belgiu...,3
2,ENTERNATIONAL,,Mar. 4/16th Agreecinent respecting the treatme...,3
3,ENTERNATIONAL,,Oct. 8th Convention between Italy and the Unit...,3
4,ARGENTINE,,June 8th Act respecting matches manufactured’ ...,3
...,...,...,...,...
152,SWITZERLAND,Tunlis,Apr. 25th Order of the Director-General of Agr...,12
153,SWITZERLAND,Tunlis,July 16th Order of the Director-General of Pub...,12
154,Unitep STATES OF AMERICA,,May 19th Act to limit the immigration of alien...,12
155,Unitep STATES OF AMERICA,,Nov. 23rd Act for the promotion of the welfare...,12


In [7]:
rec_df.loc[rec_df["page"]==7]

,country,region,text,page
71,GREAT BRITAIN,,Aug. 19th Act to provide for the reorganisatio...,7
72,GREAT BRITAIN,,Aug. 23rd Regulations made by the Secretary of...,7
73,GREAT BRITAIN,,Nov. 8th Order made by the Secretary of State ...,7
74,GREAT BRITAIN,,Nov. &th Order made by the Secretary of State ...,7
75,GREAT BRITAIN,,Nov. 8th Order made by the Secretary of State ...,7
76,GREAT BRITAIN,,Nov. roth Act to extend temporarily the period...,7
77,GREAT BRITAIN,,Nov. 15th Order of the Secretary of State exte...,7
78,GREAT BRITAIN,,Noy. 28th Regulations made by the Secretary of...,7
79,GREAT BRITAIN,,"Dec. ryth The Aerated Water Regulations, 1921,...",7
80,GREAT BRITAIN,,"Der. 29th The Hides and Skins Regulations, 192...",7


In [17]:
bins_x0, bins_x1, x0_n = group.group_line_starts_ends(lines_df)
borders = lines.make_borders_df(bins_x0, bins_x1)
display(x0_n, bins_x0, bins_x1)

3

,x0,lines,last_x0,count,page
0,145,5,145,1,1
5,199,0,199,1,1
4,223,1,223,1,1
1,"[464, 463]","[15, 16]",463,2,2
7,"[465, 485]","[6, 11]",485,2,2
2,"[535, 536]","[12, 13]",536,2,2
5,"[117, 117, 122, 122, 121, 125, 124, 127, 129, ...","[18, 19, 22, 24, 25, 26, 28, 30, 47, 50]",130,10,3
1,"[209, 212, 215, 215, 215, 215, 215, 216, 216, ...","[31, 34, 39, 45, 48, 51, 55, 58, 60, 62, 65, 6...",217,14,3
0,"[508, 509, 515, 512, 513, 513, 515, 514, 511, ...","[32, 33, 35, 36, 37, 38, 40, 41, 42, 43, 44, 4...",516,27,3
3,"[392, 390, 389, 390, 390, 390, 389, 392, 389]","[74, 83, 86, 87, 90, 92, 108, 128, 130]",389,9,4


,x1,lines,last_x1,count,page
5,1790,0,1790,1,1
4,1748,1,1748,1,1
1,1346,4,1346,1,1
3,1292,2,1292,1,1
2,1192,3,1192,1,1
...,...,...,...,...,...
2,727,603,727,1,12
3,603,595,603,1,12
0,533,622,533,1,12
8,509,580,509,1,12


In [18]:
li_df = label.assign_types(lines_df, bins_x0, bins_x1, x0_n)
l = label.assign_labels(li_df, x0_n)
l.head(20)

,line_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label
0,INTERNATIONAL LABOUR OFFICE,199,254,1790,332,1,1591,1,2,start
1,"LEGISLATIVE SERIES, 1921",223,1201,1748,1363,1,1525,2,1,end
2,(PARTS I AND ID),695,1404,1292,1469,1,597,-1,0,other
3,INDEXES,793,2040,1192,2113,1,399,-1,0,other
4,"GENEVA, SWITZERLAND",640,3143,1346,3191,1,706,-1,1,other
5,Price: 8 d.,145,3238,413,3278,1,268,0,0,country
6,INTERNATIONAL LABOUR OFFICE,465,268,1998,325,2,1533,1,1,start
7,"(GENEVA, SWITZERLAND)",908,373,1548,419,2,640,-1,0,other
8,"LEGISLATIVE SERIES, 1921",601,967,1842,1053,2,1241,-1,1,other
9,(PARTS 1 AND II),977,1080,1486,1143,2,509,-1,0,other


In [77]:
cli_df, p_l, p_g = correct_x0_types(l, bins_x0, bins_x1, x0_n)
ll_df = label.assign_labels(cli_df, x0_n)
lla_df = approve_correction(l, ll_df)

lli_df = lla_df.copy()
lli_df["new_label"] = lli_df["label"]
lli_df = label.improve_country_classification(lli_df)

lli_df.head(50)

Width median 1318.1190476190477
[(1645.0, 1), (1619.5, 2), (1267.0625, 7)]
Pages small width [7]
Pages big width [1, 2]


,line_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label,new_label
0,INTERNATIONAL LABOUR OFFICE,199,254,1790,332,1,1591,0,2,other,other
1,"LEGISLATIVE SERIES, 1921",223,1201,1748,1363,1,1525,1,1,start,start
2,(PARTS I AND ID),695,1404,1292,1469,1,597,-1,0,other,other
3,INDEXES,793,2040,1192,2113,1,399,-1,0,other,other
4,"GENEVA, SWITZERLAND",640,3143,1346,3191,1,706,2,1,end,end
5,Price: 8 d.,145,3238,413,3278,1,268,-1,0,other,other
6,INTERNATIONAL LABOUR OFFICE,465,268,1998,325,2,1533,0,1,other,other
7,"(GENEVA, SWITZERLAND)",908,373,1548,419,2,640,-1,0,other,other
8,"LEGISLATIVE SERIES, 1921",601,967,1842,1053,2,1241,2,1,end,end
9,(PARTS 1 AND II),977,1080,1486,1143,2,509,-1,0,other,other


In [78]:
l.loc[lli_df["page"]==7]

,line_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label
257,—_—~ 7 ee,899,229,1159,273,7,260,-1,1,other
258,Aug. 19th Act to provide for the reorganisatio...,270,342,1499,380,7,1229,1,2,start
259,regulation of railways and the discharge of,568,389,1496,428,7,928,2,2,middle
260,liabilities arising in connection with the pos...,568,437,1508,474,7,940,2,2,middle
261,"of railways, and otherwise to amend the Law",568,484,1496,522,7,928,2,2,middle
...,...,...,...,...,...,...,...,...,...,...
313,"Dec, 31st The Workmen’s Compensation (Twister’...",260,3023,1492,3061,7,1232,1,2,start
314,"Order, 1921, made by the Secretary of State",564,3071,1495,3108,7,931,2,2,middle
315,under section 8 of the Worknicn’s Compensation,564,3117,1493,3155,7,929,2,2,middle
316,"Act, rg06 (6 Edw. 7, c. 58)*. To. oe. eee eee ...",564,3167,1807,3204,7,1243,2,1,end


In [79]:
ll_df.loc[lli_df["page"]==7]

,line_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label
257,—_—~ 7 ee,899,229,1159,273,7,260,-1,1,other
258,Aug. 19th Act to provide for the reorganisatio...,270,342,1499,380,7,1229,1,2,start
259,regulation of railways and the discharge of,568,389,1496,428,7,928,2,2,middle
260,liabilities arising in connection with the pos...,568,437,1508,474,7,940,2,2,middle
261,"of railways, and otherwise to amend the Law",568,484,1496,522,7,928,2,2,middle
...,...,...,...,...,...,...,...,...,...,...
313,"Dec, 31st The Workmen’s Compensation (Twister’...",260,3023,1492,3061,7,1232,1,2,start
314,"Order, 1921, made by the Secretary of State",564,3071,1495,3108,7,931,2,2,middle
315,under section 8 of the Worknicn’s Compensation,564,3117,1493,3155,7,929,2,2,middle
316,"Act, rg06 (6 Edw. 7, c. 58)*. To. oe. eee eee ...",564,3167,1807,3204,7,1243,2,1,end


In [80]:
rec_gr = records.group_records(lli_df)
rec = records.extract_records(lli_df)
rec.head(20)

,country,region,text,page
0,ENTERNATIONAL,,Feb. 9th. Convention respecting insurance agai...,3
1,ENTERNATIONAL,,Feb. 14th Convention between France and Belgiu...,3
2,ENTERNATIONAL,,Mar. 4/16th Agreecinent respecting the treatme...,3
3,ENTERNATIONAL,,Oct. 8th Convention between Italy and the Unit...,3
4,ARGENTINE,,June 8th Act respecting matches manufactured’ ...,3
5,AUSTRIA,,Mar. 1oth Act respecting the regulation of emp...,3
6,AUSTRIA,,Mar. 11th Act to amend certain provisions of t...,3
7,AUSTRIA,,Mar. 22nd Act concerning the regulation of agr...,3
8,AUSTRIA,,May 11th Act relating to the contract of servi...,3
9,AUSTRIA,,June rst Act concerning the regulation of empl...,3


In [82]:
rec.loc[rec["page"]==6]

,country,region,text,page
57,FRANCE,,Ort. 30th Decree to amend the Decree of 26th A...,6
58,GERMANY,,Feb. ist Order No. 7968 to prohibit the manufa...,6
59,GERMANY,,"Feb, 2nd Order respecting the painting of ship...",6
60,GERMANY,,Feb. 5th Act respecting the works balance-shee...,6
61,GERMANY,,Apr. 11th Act to amend the law relating to acc...,6
62,GERMANY,,July zoth Act respecting maternity benefit and...,6
63,(BREMEN),,July 17th Act respecting the Council of Wage-c...,6
64,(BREMEN),,July x7th Act respecting theCouncil of Salarie...,6
65,(BREMEN),(Prussia),"Jan, 31st Order providing for (1) the amendmen...",6
66,(BREMEN),(Prussia),july rst Act to vary the rates of contribution...,6


In [18]:
rec.to_csv("index_1986.csv")

In [44]:
# correct x0_type for pages where something went wrong
def correct_x0_types(lines_df, bins_x0, bins_x1, x0_n):
    df = lines_df.copy()
    start_page = lines_df["page"].min()

    text_widths = list(lines.make_borders_df(bins_x0, bins_x1)["dx"]) # difference between mean of first and last bin for x0 for every page

    tw = text_widths.copy()
    tw.sort()
    width_median = tw[int(len(tw)/2)]
    print("Width median", width_median)

    f = filter(lambda w: (not util.similar_to(w, width_median, 10)), text_widths) # filter widths that differ from the rest
    f = list(f)

    p = []
    for w in range(len(f)):
        p.append((f[w], text_widths.index(f[w]) + start_page)) # add page to strange width
    
    print(p)

    p_l = [a for w, a in p if w<width_median]
    p_g = [a for w, a in p if w>width_median]
    
    print("Pages small width", p_l)
    print("Pages big width", p_g)

    df.loc[df["page"].isin(p_l) & (df["x0_type"]>=0), "x0_type"] +=1 # correct wrong x0_type for p_l

    # correct wrong x0_type (and x1_type) for p_g
    bins = group.get_line_start_end_bins(df.loc[df["page"].isin(p_g)])
    bins_x0_cor = group.get_relevant_x0_bins(bins[0], x0_n, drop_first=True) # drop bin on the very left
    df_cor = label.assign_types(df.loc[df["page"].isin(p_g)], bins_x0_cor, bins_x1.loc[bins_x1["page"].isin(p_g)], x0_n)
    df.loc[df["page"].isin(p_g), ["x0_type", "x1_type"]] = df_cor[["x0_type", "x1_type"]]

    return df, p_l, p_g

In [74]:
def approve_correction(orig_df, cor_df):
    df = cor_df.copy()

    start_counts = []
    for p, frame in orig_df.loc[df["page"].isin(p_l)].groupby("page"):
        c = frame.loc[frame["label"]=="start"].shape[0]
        start_counts.append(c)

    start_counts_c = []
    for p, frame in cor_df.loc[df_c["page"].isin(p_l)].groupby("page"):
        c = frame.loc[frame["label"]=="start"].shape[0]
        start_counts_c.append(c)

    for i in range(len(start_counts)):
        if (start_counts_c[i] <= 2) & (start_counts_c[i] < start_counts[i]): # after correction significantly less start lines
            cor_df.loc[cor_df["page"]==p_l[i]] = orig_df.loc[orig_df["page"]==p_l[i]]

    return cor_df

In [17]:
for i, file in enumerate(indexes_csv):
    save_path = output_dir + os.sep + os.path.basename(file)
    extract.extract_indexes(pd.read_csv(file), save_to=save_path)

Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1976.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1977.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1978.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1979.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1981.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1982.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_Index_1983.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1983_Spanish.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1984.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_year

## Probleme
- Dokumente mit 2 Spalten
- Dokumente mit Länder-Überschrift über ganze Breite der Seite
- Länder- und Regionenspalte vereinen?
    - Regionenerkennung oft nicht richtig, weil Groß- und Kleinbuchstaben vom OCR nicht richtig erkannt werden
- Dokumente/Seiten mit vielen Artifakten
    - 1944 seiten 8-9, schlechte Ländererkennung
- 1921 Seite 7 FIXED
- 1986 Seiten 4-7 FIXED

## Fragen
- Kommandozeilenprogramm
